In [ ]:
import random as rd
import numpy as np
import time

PATH = 0
HIGHLIGHTS = 1

クラス群

In [ ]:
class Grid():
  def __init__(self, stay, attr, e_bool):
    self.s_dist = stay # 滞在時間分布を設定
    self.signal = [] # このグリッドで受けるビーコンの電波のリストを初期化
    self.attribute = attr # 通路か売場か見分ける
    self.entrance = e_bool # 入り口かどうか

  def set_trans_dist(self, trans):
    self.t_dist = trans # 移動確率分布を設定[[1, 0]方向, [-1, 0]方向, [0, 1]方向, [0, -1]方向]

  def set_signal(self, unitid, rssi):
    self.signal.append([unitid, rssi])

class Person():
  def __init__(self, id, x, y, time, staytime, num, random_ratio):
    self.id = id # userid
    self.ord = [x, y] # 現在地の座標
    self.random = rd.choices([0, 1], weights=[1, random_ratio])[0] # random flag
    self.p_num = num # 不変の番号を一応ふっておく
    self.probe = rd.choice([3, 5, 10]) # プローブ要求間隔のバリエーション（どんなのがあるかわからん）
    self.next_probe = time + + rd.randrange(self.probe) # 次にプローブ要求を発する時間
    self.next_move = time + 1 + staytime # 次に動く時間
    self.entry_time = time # グリッド上に現れた時間
    # self.group = 0 # 誰かと一緒にいるか？

  def probe_reset(self, dist): # 次にプローブ要求を発する時間
    if dist == 0:
      self.next_probe += self.probe
    else:
      self.next_probe += self.probe + rd.gauss(dist[0], dist[1])

  def move(self, vari): # 人の座標移動
    self.ord[0] += vari[0]
    self.ord[1] += vari[1]

  def move_time(self, staytime): # 次に動く時間
    self.next_move += 1 + staytime

  def id_update(self, id): # randomizedの場合のuseridの更新
    self.id = id

  def pre_move(self, pre): # ひとつ前の座標移動
    self.pre = pre


シミュレーション

In [ ]:
# (サンプル数, グリッドサイズ, ビーコンの位置, 入口の位置, 売場の位置, ランダムアドレスの端末数/固定アドレスの端末数, 
# プローブ要求のノイズ(0ならノイズなし, [μ, σ]なら正規分布によるノイズ), 初期人数([[x座標, y座標, 人数], ....]), 
# 移動確率の比[通路, 売場], 滞在時間分布がポアソン分布に従うとしたときのλ[通路, 売り場], 
# タイムスタンプごと入口から入ってくる人数がポアソン分布に従うとしたときのλ, 入口から出ていく確率, 
# ひとつ前の状態に戻る行動を選ぶ確率への割引率, シード値)
def simulator(
    n_samples=100, mapsize=[5, 5], pos_beacons=[[0, 0], [3, 3]], pos_entrances=[[0, 0]],
    pos_highlights=[[0, 3], [0, 4], [2, 2], [2, 3], [3, 2], [3, 3], [4, 0]], random_ratio=0.5,
    probe_noise=0, init_people=[[0, 0, 1], [4, 4, 1]], trans_ratio=[1, 1.5], stay=[1, 3],
    leave=0.9, enter=0.3, move_discount=0.1, seed=None):
  n_beacons = len(pos_beacons)
  n_entrances = len(pos_entrances)
  for pos in pos_beacons:
    if not (0 <= pos[0] < mapsize[0] and 0 <= pos[1] < mapsize[1]):
      print("beacons must be on grids")
      return -1
  for pos in pos_entrances:
    if not (0 <= pos[0] < mapsize[0] and 0 <= pos[1] < mapsize[1]):
      print("entrances must be on grids")
      return -1
  for pos in pos_highlights:
    if not (0 <= pos[0] < mapsize[0] and 0 <= pos[1] < mapsize[1]):
      print("highlights must be on grids")
      return -1

  np.random.seed(seed)
  rd.seed(seed)

  # ビーコンからの距離によってrssiを決定
  def grid_signal(distance): # v0では簡単な式で表す
    return distance*10

  def rssi_range(rssi):
    # rssiの値によってrangeを決定
    if rssi <= 70:
      range = 1
    elif rssi <= 80:
      range = 2
    elif rssi <= 85:
      range = 3
    elif rssi <= 90:
      range = 4
    elif rssi <= 95:
      range = 5
    else:
      range = 9

    return range 

  sigrange  = 5 # 電波の届くマス目
  # ビーコンを設置し各グリッドにrssiを設定
  def set_beacon(x, y, unitid, map, mapsize):
    for i in range(sigrange+1):
      for a in range(-i, i+1):
        absb = abs(i) - abs(a)
        if 0 <= x+a < mapsize[0]:
          if 0 <= y+absb < mapsize[1]:
            map[x+a][y+absb].set_signal(unitid, grid_signal(i))
          if absb != 0 and 0 <= y-absb < mapsize[1]:
            map[x+a][y-absb].set_signal(unitid, grid_signal(i))

  def gen_id(id_list):
    while 1:
      id = format(rd.randrange(16**8), '08x') + '-' + format(rd.randrange(16**4), '08x') + '-' + format(rd.randrange(16**4), '08x') + '-' + format(rd.randrange(16**4), '08x') + '-' + format(rd.randrange(16**12), '08x')
      if id_list == []:
        break
      elif id not in id_list:
        break
    return id

  def movable(grid):
    pre_move_list = [] # ひとつ前の行動のうち選択して[n, m]に移動してこれるもの
    if grid.entrance:
      pre_move_list.append([-2, -2]) # 入り口から入ってきた
    if grid.t_dist[1] != 0: # [-1, 0]に動けるなら[1, 0]の行動でこのマスに来れる
      pre_move_list.append([1, 0])
    if grid.t_dist[0] != 0:
      pre_move_list.append([-1, 0])
    if grid.t_dist[3] != 0:
      pre_move_list.append([0, 1])
    if grid.t_dist[2] != 0:
      pre_move_list.append([0, -1])
    pre = rd.choice(pre_move_list)
    return pre

  # マップを作る
  i = 0
  map = [] # マップを表すリストを初期化
  while i < mapsize[0]:
    j = 0
    line = [] # 1行分のリストを初期化
    while j < mapsize[1]:
      if [i, j] in pos_highlights:
        attr = HIGHLIGHTS
      else:
        attr = PATH
      if [i, j] in pos_entrances:
        ent = True
      else:
        ent = False
      grid = Grid(stay[attr], attr, ent)
      line.append(grid)
      j += 1
    map.append(line) # マップのリストに1行分追加
    i += 1

  i = 0
  while i < mapsize[0]:
    j = 0
    while j < mapsize[1]:
      trans = []
      if i+1 >= mapsize[0]:
        trans.append(0)
      elif map[i+1][j].attribute == PATH:
        trans.append(trans_ratio[0])
      else:
        trans.append(trans_ratio[1])
      
      if i-1 < 0:
        trans.append(0)
      elif map[i-1][j].attribute == PATH:
        trans.append(trans_ratio[0])
      else:
        trans.append(trans_ratio[1])

      if j+1 >= mapsize[1]:
        trans.append(0)
      elif map[i][j+1].attribute == PATH:
        trans.append(trans_ratio[0])
      else:
        trans.append(trans_ratio[1])

      if i-1 < 0:
        trans.append(0)
      elif map[i][j-1].attribute == PATH:
        trans.append(trans_ratio[0])
      else:
        trans.append(trans_ratio[1])
      map[i][j].set_trans_dist(trans) # 各方向への移動確率を設定
      j += 1
    i += 1
    
  # ビーコンを生成しその情報を辞書型に記憶
  x = pos_beacons[0][0]
  y = pos_beacons[0][1]
  unitid = format(rd.randrange(16**8), '08x').upper()
  beacons = {"%d, %d" % (x, y):unitid}
  set_beacon(x, y, unitid, map, mapsize)

  i = 1
  while i < n_beacons:
    x = pos_beacons[i][0]
    y = pos_beacons[i][1]

    while 1:
      unitid = format(rd.randrange(16**8), '08x').upper()
      if unitid not in beacons.values():
        break
    
    beacons["%d, %d" % (x, y)] = unitid
    set_beacon(x, y, unitid, map, mapsize)
    i += 1

  if seed == None:
    timestamp = int(time.time())
  else:
    timestamp = 1594911600  # 再現性のために固定したほうがいい？

  num = 0
  people = []
  id_list = []
  # Personを生成し初期位置に配置
  if type(init_people[0]) == list:
    for ip in init_people:
      n_people = ip[2]
      for p in range(n_people):
        num += 1
        id = gen_id(id_list)
        id_list.append(id)
        dist = grid.s_dist
        # staytime = abs(int(rd.choices([rd.gauss(dist[0][0], dist[1][0]), rd.gauss(dist[0][1], dist[1][1])], weights=dist[2])[0]))
        staytime = np.random.poisson(dist)
        person = Person(id, ip[0], ip[1], timestamp, staytime, num, random_ratio)
        people.append(person)
        person.pre_move(movable(map[ip[0]][ip[1]]))
  else:
    for n in range(mapsize[0]):
      for m in range(mapsize[1]):
        n_people = np.random.poisson(init_people[map[n][m].attribute]) # 各グリッドの人数の初期値
        for p in range(n_people):
          num += 1
          id = gen_id(id_list)
          id_list.append(id)
          dist = grid.s_dist
          # staytime = abs(int(rd.choices([rd.gauss(dist[0][0], dist[1][0]), rd.gauss(dist[0][1], dist[1][1])], weights=dist[2])[0]))
          staytime = np.random.poisson(dist)
          person = Person(id, n, m, timestamp, staytime, num, random_ratio)
          people.append(person)
          person.pre_move(movable(map[n][m]))

  # timestampごとに人が移動
  data = []
  while 1:
    # データ生成
    for p in people:
      if p.next_probe < timestamp:
        p.probe_reset(probe_noise)
        ord = p.ord
        if p.random == 1:
          id = gen_id(id_list)
          id_list.append(id)
          p.id_update(id)
        for b in map[ord[0]][ord[1]].signal:
          column = [p.id, str(rssi_range(b[1])), str(b[1]), str(p.random), str(timestamp), b[0]]
          data.append(column)
          if len(data) >= n_samples:
            break

    if len(data) >= n_samples:
      # print("length: " + str(len(data)))
      break
    
    # 動き
    for p in people:
      if p.next_move == timestamp:

        if p.ord in pos_entrances:
          if p.pre == [-2, -2]:
            leave *= move_discount # 入ってきたばっかりの人が出ていく確率はだいぶ減らしたい？
            
          if rd.choices([0, 1], weights=[1-leave, leave])[0]:
            people.remove(p) # 入口から出ていく
            continue

        tmp_trans = map[p.ord[0]][p.ord[1]].t_dist
        if p.pre == [-1, 0]:
          tmp_trans[0] *= move_discount
        elif p.pre == [1, 0]:
          tmp_trans[1] *= move_discount
        elif p.pre == [0, -1]:
          tmp_trans[2] *= move_discount
        elif p.pre == [0, 1]:
          tmp_trans[3] *= move_discount

        variation = rd.choices([[1, 0], [-1, 0], [0, 1], [0, -1]], weights=tmp_trans)[0]
        p.move(variation)
        p.pre_move(variation)
        dist = map[p.ord[0]][p.ord[1]].s_dist
        # staytime = abs(int(rd.choices([rd.gauss(dist[0][0], dist[1][0]), rd.gauss(dist[0][1], dist[1][1])], weights=dist[2])[0]))
        staytime = np.random.poisson(dist)
        p.move_time(staytime)

    # 入口から入ってくる人
    for e in pos_entrances:
      # n_enter = abs(int(rd.gauss(enter[0], enter[1])))
      n_enter = np.random.poisson(enter)
      for p in range(n_enter):
        id = gen_id(id_list)
        id_list.append(id)
        dist = map[e[0]][e[1]].s_dist
        # staytime = abs(int(rd.choices([rd.gauss(dist[0][0], dist[1][0]), rd.gauss(dist[0][1], dist[1][1])], weights=dist[2])[0]))
        staytime = np.random.poisson(dist)
        num += 1
        person = Person(id, e[0], e[1], timestamp, staytime, num, random_ratio)
        people.append(person)
        person.pre_move([-2, -2])

    timestamp += 1

  return data
  


In [ ]:
simulator()

[['caf096aa-00007232-0000de1c-0000f824-72a571b686c3',
  '1',
  '20',
  '0',
  '1602486555',
  '5ACC521A'],
 ['3e600ac6-00009d9d-000085b8-00001796-d33703a2463',
  '1',
  '0',
  '1',
  '1602486555',
  '7A9B335C'],
 ['caf096aa-00007232-0000de1c-0000f824-72a571b686c3',
  '1',
  '10',
  '0',
  '1602486558',
  '5ACC521A'],
 ['6b419aca-0000c5b6-00002222-000076a7-63484eaddcaf',
  '1',
  '0',
  '0',
  '1602486558',
  '7A9B335C'],
 ['44aaa5c2-0000b47f-00004fe2-00004cdc-e8842a54a295',
  '1',
  '10',
  '0',
  '1602486559',
  '7A9B335C'],
 ['44aaa5c2-0000b47f-00004fe2-00004cdc-e8842a54a295',
  '1',
  '50',
  '0',
  '1602486559',
  '5ACC521A'],
 ['caf096aa-00007232-0000de1c-0000f824-72a571b686c3',
  '1',
  '50',
  '0',
  '1602486561',
  '7A9B335C'],
 ['caf096aa-00007232-0000de1c-0000f824-72a571b686c3',
  '1',
  '30',
  '0',
  '1602486561',
  '5ACC521A'],
 ['6b419aca-0000c5b6-00002222-000076a7-63484eaddcaf',
  '1',
  '20',
  '0',
  '1602486561',
  '7A9B335C'],
 ['6b419aca-0000c5b6-00002222-000076a7-6